# Import relevant libraries

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import obspy
from obspy.taup import TauPyModel,taup_create
%matplotlib inline

In [6]:
%%sh
subl ../input_prep/inputs/target_event.txt
subl ../input_prep/inputs/puna_catalog_hypodd_v5.txt


# 2. Prepare the Input Files

Read the station file, target event, corresponding phase file and prepare the focmec_prelim file and polarity file.

In [8]:
#Extract the phase file for the event
fi=open('../input_prep/inputs/phase_input_hypodd_v5.txt','r')
line=fi.readlines()

event="2008_12_04_14_3" #Edit it for each event

inpdate=event.strip()
yrs,mns,dys,hrs,mms=inpdate.split("_")

eveid="%s%s%s%s%s"%(yrs,mns,dys,hrs,mms)
print(eveid)

fo=open("../input_prep/phasefiles_v5/%s%s%s%s%s_phase_velest_v5.txt"%(yrs,mns,dys,hrs,mms),'w')	
flag=0
for i in line:
    if i.split()[0]=="#":
        flag=0
        jj,yr,mn,dy,hr,mm,sc,lat,lon,dep,k,l,m,res,num=i.split()

        if (yr==yrs and mn==mns and dy==dys and hr==hrs and mm==mms):
            flag=1
            print(flag)
    if flag==1 and i.split()[0]!="#":
        net=i.split()[0][0:2]
        stn=i.split()[0][2:6]
        if stn=="GALA":
            stn="GALAN"
        print(net,stn,i.split()[1],i.split()[2],i.split()[3],file=fo)		

20081204143
1


In [10]:
#Prepare the focmec_prelim file and polarity file:

df1=pd.read_csv("../input_prep/inputs/spuna_stations.txt",sep="\s+",header=None) #Station file
df3=pd.read_csv("../input_prep/inputs/target_event.txt",sep="\s+",header=None) #Event file
df2=pd.read_csv("../input_prep/phasefiles_v5/%s_phase_velest_v5.txt"%eveid,sep="\s+",header=None) #Phase file

print(eveid)
evelat=float(df3[6][0])
evelon=float(df3[7][0])
evedep=float(df3[8][0])+3.1
print(evelat,evelon,evedep)
    
#Velocity model to use
taup_create.build_taup_model("../input_prep/inputs/punamodelobspy.nd", output_folder='velmodel')
#Velocity model: Add full path here
model = TauPyModel(model="/Users/sankha/Desktop/research/southern_puna/focmec/input_prep/velmodel/punamodelobspy")

#Output files
fo=open("../input_prep/prelims_v5/focmec_prelim_%s_p.txt"%eveid,'w') #Detailed Output file
fo3=open("../input_prep/polarity_v5/%s_p.txt"%eveid,'w') #Initial polarity file
for i in range(len(df2[0])): #Phase file
    for j in range(len(df1[0])): #Station file
        if df2[4][i]=="P" and df2[0][i]==df1[2][j] and df2[1][i]==df1[3][j]:

            #Distance, azimuth and backazimuth
            dist,az,baz=obspy.taup.taup_geo.calc_dist_azi(evelat,evelon,df1[1][j],df1[0][j] ,6378,0)
            #Arrival time and take-off angle
            #In the velocity model, 0 depth= avg. station elevation, so depths are scaled by avg. station elev.
            arrivals = model.get_travel_times(source_depth_in_km=evedep,distance_in_degree=dist)
            
            #print network, station, tt, phase, lat, lon, elev, dist
            
            print(df2[0][i],df2[1][i],df2[2][i],df2[4][i],df1[0][j],df1[1][j],df1[5][j],dist,az,baz,arrivals[0].time,arrivals[0].takeoff_angle,file=fo)                                 
            print(df2[0][i],df2[1][i],df2[4][i],"U","IP",file=fo3)
                
                

20081204143
-26.484288 -67.541467 13.958
Building obspy.taup model for '../input_prep/inputs/punamodelobspy.nd' ...


### Create focmec input file:

In [12]:
inps=pd.read_csv("../input_prep/prelims_v5/focmec_prelim_%s_p.txt"%eveid,sep='\s+',header=None) #Detailed Output file
pol=pd.read_csv("../input_prep/polarity_v5/%s_p.txt"%eveid,sep='\s+',header=None)

#Output files
fo2=open("../input_prep/focmec_input/puna_%s_p.inp"%eveid,'w') #Focmec input file
print(eveid,file=fo2)
for i in range(len(pol[0])):
    print("%s%8.2f%8.2f%s%20s"%(inps[1][i],inps[8][i],inps[11][i],pol[3][i],pol[4][i]),file=fo2)
print(eveid)

20081204143


In [13]:
%%sh
e="20081204143" #eventid

#Create a run folder
cp -r ../focmec_runs/template ../focmec_runs/$e

#Copy the input file to the folder
cp ../input_prep/focmec_input/puna_${e}_p.inp ../focmec_runs/$e/puna_${e}_p.inp

subl ../focmec_runs/$e/puna_${e}_p.inp

## 3. Pick Polarity from Data

In [14]:
event="2008_12_04_14_3" #Edit this

inpdate=event.strip()
yrs,mns,dys,hrs,mms=inpdate.split("_")

eveid="%s%s%s%s%s"%(yrs,mns,dys,hrs,mms)
print(eveid)

20081204143


In [15]:
print(eveid)
evelst=pd.read_csv("../input_prep/inputs/target_event.txt",sep="\s+",header=None)
#read the distance and azimuth
df=pd.read_csv("../input_prep/prelims_v5/focmec_prelim_%s_p.txt"%eveid,sep='\s+',header=None)
net=np.array(df[0])
stn=np.array(df[1])
ptime=np.array(df[2])
azlist=np.array(df[8])
dist=np.array(df[7])

20081204143


In [ ]:
#Edit the file path and pick the polarity
i=0
tii="%d-%02d-%02dT%02d:%02d:%4.2f"%(evelst[0][i],evelst[1][i],evelst[2][i],evelst[3][i],evelst[4][i],evelst[5][i])
print(tii)
#Picking
dt1=4
dt2=4
for i in range(len(df[0])):
    ti=obspy.UTCDateTime(tii)
    st1=obspy.read("../data/20081204/2008_12_04_000000.00_%s_%s_mseed"%(net[i],stn[i]), starttime=ti+ptime[i]-dt1, endtime=ti+ptime[i]+dt2)
    st2=st1.select(component='Z')
    st2.detrend("spline", order=3, dspline=500)
    st2.filter("bandpass", freqmin=0.5, freqmax=15)
    st2.plot(type='relative',grid_color="white",linewidth=0.5,block=False)